# QAOA Warm-Start GPU: Portfolio Optimization with Heuristic Initialization

**Objective**: Implement QAOA for portfolio optimization using warm-start initializations from **classical financial heuristics**, not from the optimal solution.

## What are Heuristic Warm-Starts?

Instead of initializing the quantum state randomly, we start from **reasonable but sub-optimal solutions** obtained with classical greedy methods:

1. **Greedy-Return**: Select the B assets with the highest individual expected return
2. **Greedy-Risk**: Select the B assets with the lowest individual volatility
3. **Greedy-Sharpe**: Select the B assets with the best individual Sharpe ratio

**Why are these heuristics NOT optimal?**

* They ignore **correlations** between assets (diversification)
* They do not consider the full **covariance matrix**
* They are myopic (greedy) approximations without a global view

**Objective of QAOA**: Start from these sub-optimal solutions (~60–80% gap) and explore the quantum space to **find better solutions or ones close to the global optimum**.

---

## Notebook Features

✅ **3 heuristic warm-starts** (without using brute-force — that would be cheating)

✅ **XY mixer** to preserve cardinality

✅ **GPU acceleration** (if available)

✅ **Honest benchmark** vs brute-force to measure real improvement

---


In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.circuit.library import RXXGate, RYYGate, RZZGate
from qiskit_aer import AerSimulator

from scipy.optimize import minimize

import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ All libraries imported")


✓ All libraries imported


## 1. Configuration


In [24]:
CONFIG = {
    'p_layers': 6,
    'shots': 8000,
    'seed': 7,
    'n_warm_starts': 3,
    'optimizer': 'COBYLA',
    'max_iter': 250,
    'rhobeg': 0.5,
    'gpu_enabled': True
}

P_LAYERS = CONFIG['p_layers']
SHOTS = CONFIG['shots']
SEED = CONFIG['seed']

rng = np.random.default_rng(SEED)

print("="*55)
print("QAOA - ONLY WARM-STARTS (NO RANDOM-STARTS)")
print("="*55)
for key, val in CONFIG.items():
    print(f"  {key:21s}: {val}")
print("="*55)


QAOA - ONLY WARM-STARTS (NO RANDOM-STARTS)
  p_layers             : 6
  shots                : 8000
  seed                 : 7
  n_warm_starts        : 3
  optimizer            : COBYLA
  max_iter             : 250
  rhobeg               : 0.5
  gpu_enabled          : True


In [25]:
# ============================================================================
# FETCH AND PREPARE DATA (from Data and QUBO.ipynb)
# ============================================================================

data = np.load("data\portfolio_qubo_data.npz", allow_pickle=True)

Q = data['Q']
q = data['q']
mu = data['mu']
Sigma = data['Sigma']
B = int(data['B'])
TICKERS = list(data['TICKERS'])
n = len(TICKERS)

print("✓ Data loaded successfully.")
print(f"  n = {n} assets")
print(f"  B = {B} cardinality")
print(f"  Q shape: {Q.shape}")
print(f"  q shape: {q.shape}")

✓ Data loaded successfully.
  n = 21 assets
  B = 4 cardinality
  Q shape: (21, 21)
  q shape: (21,)


In [33]:
# ============================================================================
# HELPER FUNCTIONS (Defined in QAOA_functions.qaoa_utilities)
# ============================================================================

from QAOA_functions.qaoa_utilties import build_qaoa_xy, f_qubo, qubo_to_ising
from QAOA_functions.qaoa_utilties import bitarray_from_qiskit_string, bind_params

def is_valid(x, B):
    return np.sum(x) == B

J, h, const_shift = qubo_to_ising(Q, q, n)
print(f"✓ QUBO→Ising transformation complete")

✓ QUBO→Ising transformation complete


## Warm-Start Initializations: Financial Heuristics

We generate 3 initial portfolios using simple greedy strategies that **DO NOT require quadratic optimization**:

### 1. Greedy-Return (Maximum Return)

* Criterion: Select the top B assets with the highest $\mu_i$
* Logic: "Invest in those that earn the most"
* Issue: Ignores risk and correlations
* Expected gap vs optimum: ~60–80%

### 2. Greedy-Risk (Minimum Risk)

* Criterion: Select the top B assets with the lowest $\sigma_i^2$
* Logic: "Invest in the most stable ones"
* Issue: Ignores return and diversification
* Expected gap vs optimum: ~70–90%

### 3. Greedy-Sharpe (Best Individual Ratio)

* Criterion: Select the top B assets with the highest $\mu_i / \sigma_i$
* Logic: "Invest in those with the best risk-adjusted return"
* Issue: Individual Sharpe ≠ portfolio Sharpe
* Expected gap vs optimum: ~50–70%

**Important note**: None of these heuristics considers the full covariance matrix $\Sigma$, therefore they are **guaranteed to be sub-optimal** for the true QUBO problem.

QAOA will attempt to **improve these solutions** by exploring the Hilbert space.


In [27]:
initial_states = []
labels = []

# Warm-start 1: greedy máximo retorno esperado
returns_sorted = np.argsort(-mu)
greedy_return = np.zeros(n, dtype=int)
greedy_return[returns_sorted[:B]] = 1
initial_states.append(greedy_return)
labels.append("Greedy-Return")
print("✓ Warm-start 1: greedy max return")
print("  Assets:", [TICKERS[i] for i in returns_sorted[:B]])
print("  Cost:", f_qubo(greedy_return, Q, q))

# Warm-start 2: greedy mínimo riesgo individual
risks_sorted = np.argsort(np.diag(Sigma))
greedy_risk = np.zeros(n, dtype=int)
greedy_risk[risks_sorted[:B]] = 1
initial_states.append(greedy_risk)
labels.append("Greedy-Risk")
print("\n✓ Warm-start 2: greedy min risk")
print("  Assets:", [TICKERS[i] for i in risks_sorted[:B]])
print("  Cost:", f_qubo(greedy_risk, Q, q))

# Warm-start 3: greedy máximo Sharpe ratio individual
# Sharpe individual aproximado: mu_i / sqrt(Sigma_ii)
sharpe_individual = mu / np.sqrt(np.diag(Sigma) + 1e-10)
sharpe_sorted = np.argsort(-sharpe_individual)
greedy_sharpe = np.zeros(n, dtype=int)
greedy_sharpe[sharpe_sorted[:B]] = 1
initial_states.append(greedy_sharpe)
labels.append("Greedy-Sharpe")
print("\n✓ Warm-start 3: greedy max Sharpe")
print("  Assets:", [TICKERS[i] for i in sharpe_sorted[:B]])
print("  Cost:", f_qubo(greedy_sharpe, Q, q))

print(f"\nTotal initializations: {len(initial_states)} (all heuristic warm-starts)")



✓ Warm-start 1: greedy max return
  Assets: ['WMT', 'JNJ', 'AAPL', 'NVDA']
  Cost: 0.8145007198741395

✓ Warm-start 2: greedy min risk
  Assets: ['AVGO', 'CAT', 'UNH', 'TSLA']
  Cost: 0.16148141876589922

✓ Warm-start 3: greedy max Sharpe
  Assets: ['JNJ', 'AAPL', 'WMT', 'NVDA']
  Cost: 0.8145007198741395

Total initializations: 3 (all heuristic warm-starts)


In [28]:
print("Setting up backend...")
try:
    if CONFIG['gpu_enabled']:
        try:
            backend = AerSimulator(device='GPU', method='statevector')
            test_qc = QuantumCircuit(2)
            test_qc.h(0)
            test_qc.measure_all()
            backend.run(test_qc, shots=10).result()
            print("✓ GPU backend initialized")
        except:
            backend = AerSimulator(method='statevector')
            print("✓ CPU backend initialized (no GPU available)")
    else:
        backend = AerSimulator(method='statevector')
        print("✓ CPU backend initialized")
except:
    backend = AerSimulator(method='statevector')
    print("✓ CPU backend initialized")


Setting up backend...
✓ CPU backend initialized (no GPU available)


In [29]:
def get_objective(ansatz, theta_params, t_ansatz):
    qaoa_trace = []
    iteration_count = [0]
    def objective(theta):
        param_dict = {p: float(t) for p, t in zip(theta_params, theta)}
        circ = bind_params(t_ansatz, param_dict)
        result = backend.run(circ, shots=SHOTS, seed_simulator=SEED).result()
        counts = result.get_counts()
        total_cost = 0.0
        valid = 0
        for bitstring, count in counts.items():
            x = bitarray_from_qiskit_string(bitstring)
            if is_valid(x, B):
                total_cost += count * f_qubo(x, Q, q)
                valid += count
        if valid == 0:
            return qaoa_trace[-1] if qaoa_trace else 0.0
        avg_cost = total_cost / valid
        qaoa_trace.append(avg_cost)
        iteration_count[0] += 1
        if iteration_count[0] % 50 == 0:
            print(f"    Iter {iteration_count[0]:4d}: cost = {avg_cost:.6f}, valid = {valid}/{SHOTS}")
        return avg_cost
    return objective


In [30]:
print("\n" + "="*60)
print("QAOA: ONLY WARM-STARTS")
print("="*60)
results = []
best_val = np.inf
best_theta = None
best_idx = None
t_total_start = time.perf_counter()

for i, (init_bits, label) in enumerate(zip(initial_states, labels)):
    print(f"\n[{i+1}/3] WARM-START ({label})")
    ansatz, theta_params = build_qaoa_xy(n, P_LAYERS, J, h, init_bits)
    ansatz_meas = ansatz.copy()
    ansatz_meas.measure_all()
    t_ansatz = transpile(ansatz_meas, backend, optimization_level=1, seed_transpiler=SEED)
    objective = get_objective(ansatz, theta_params, t_ansatz)
    x0 = rng.uniform(0, 2*np.pi, size=2*P_LAYERS)
    opt_result = minimize(
        objective, x0, method='COBYLA',
        options={'maxiter': CONFIG['max_iter'], 'rhobeg': CONFIG['rhobeg']}
    )
    print(f"  Final cost: {opt_result.fun:.6f} ({opt_result.nfev} evals)")
    results.append({
        'label': label,
        'final_cost': opt_result.fun,
        'final_theta': opt_result.x,
        'nfev': opt_result.nfev,
        'ansatz': ansatz,
        'theta_params': theta_params,
        'init_bits': init_bits.copy()
    })
    if opt_result.fun < best_val:
        best_val = opt_result.fun
        best_theta = opt_result.x
        best_idx = i

t_total_end = time.perf_counter()
print(f"\nAll optimizations complete in {t_total_end-t_total_start:.2f}s")
print(f"Best found at idx={best_idx+1} ({results[best_idx]['label']}) with cost {best_val:.6f}")

best_ansatz = results[best_idx]['ansatz']
best_theta_params = results[best_idx]['theta_params']



QAOA: ONLY WARM-STARTS

[1/3] WARM-START (Greedy-Return)


    Iter   50: cost = 0.279377, valid = 8000/8000
    Iter  100: cost = 0.267905, valid = 8000/8000
  Final cost: 0.267421 (148 evals)

[2/3] WARM-START (Greedy-Risk)
    Iter   50: cost = 0.338707, valid = 8000/8000
    Iter  100: cost = 0.327228, valid = 8000/8000
  Final cost: 0.325965 (142 evals)

[3/3] WARM-START (Greedy-Sharpe)
    Iter   50: cost = 0.284456, valid = 8000/8000
    Iter  100: cost = 0.281432, valid = 8000/8000
  Final cost: 0.280506 (141 evals)

All optimizations complete in 1925.67s
Best found at idx=1 (Greedy-Return) with cost 0.267421


In [31]:
print("\nSampling with best found parameters...")

best_init_bits = results[best_idx]['init_bits']
best_circuit, best_theta_params = build_qaoa_xy(n, P_LAYERS, J, h, best_init_bits)
circuit_measured = best_circuit.copy()
circuit_measured.measure_all()
t_trans = transpile(circuit_measured, backend, optimization_level=1, seed_transpiler=SEED)
final_params = {p: float(t) for p, t in zip(best_theta_params, best_theta)}
final_circ = bind_params(t_trans, final_params)

res = backend.run(final_circ, shots=SHOTS, seed_simulator=SEED).result()
counts = res.get_counts()

valid_solutions = []
for bitstring, count in counts.items():
    x = bitarray_from_qiskit_string(bitstring)
    if is_valid(x, B):
        cost = f_qubo(x, Q, q)
        valid_solutions.append((bitstring, count, cost, x))

valid_solutions.sort(key=lambda t: t[2])
s_best, c_best, fx_best, x_best = valid_solutions[0]
sel_idx = np.where(x_best == 1)[0]
sel_tickers = [TICKERS[i] for i in sel_idx]

w = np.zeros(n)
w[sel_idx] = 1.0 / B
mu_ann = 252 * float(mu@w)
std_ann = np.sqrt(252 * float(w@Sigma@w))
sharpe = mu_ann / std_ann if std_ann > 1e-6 else 0.0

print(f"\n{'='*60}")
print("        FINAL QAOA WARM-START RESULTS")
print(f"{'='*60}")
print(f"Bitstring: {s_best}  QUBO cost: {fx_best:.6f}")
print("Portfolio:", ", ".join(sel_tickers))
print(f"Annualized Return: {mu_ann*100:.2f}%   Volatility: {std_ann*100:.2f}%   Sharpe: {sharpe:.3f}")



Sampling with best found parameters...

        FINAL QAOA WARM-START RESULTS
Bitstring: 011000010000000100000  QUBO cost: 0.151617
Portfolio: AVGO, UNH, CAT, VZ
Annualized Return: 3.28%   Volatility: 17.28%   Sharpe: 0.190


In [32]:
try:
    bf = np.load("data\\bruteforce_results.npz", allow_pickle=True)
    fx_bf = float(bf['fx_bruteforce'])
    gap = fx_best - fx_bf
    gap_r = 100*gap/fx_bf if fx_bf != 0 else np.nan
    print("\nComparison to brute-force:")
    print(f"  QAOA cost:  {fx_best:.6f}")
    print(f"  BF  cost:   {fx_bf:.6f}")
    print(f"  Absolute gap: {gap:.6f}")
    print(f"  Relative gap: {gap_r:.2f}%")
except Exception as e:
    print("Brute-force data not loaded:", e)



Comparison to brute-force:
  QAOA cost:  0.151617
  BF  cost:   0.147206
  Absolute gap: 0.004411
  Relative gap: 3.00%
